In [14]:
import numpy as np
import random as nr
import copy

In [15]:
with open('sample dataset.txt','r') as reader:
    #open file and get data in a list and final close
    data_list = reader.readlines()

node_number = int(data_list[0])

#create adjacency list 
adj_list = [[] for i in range(node_number)]

#fill list with data
for i in range(1,len(data_list)):

    this = data_list[i].split(" ")
    row = int(this[0])-1
    col = int(this[1])-1     
    adj_list[row].append(col)
    adj_list[col].append(row)

edge_number = len(data_list)-1

In [16]:
def generate_chromosome(adj_list):
    this = []
    for i in range(node_number):
        if len(adj_list[i])!=1:
            select_random = nr.randint(0,len(adj_list[i])-1)
            this.append([adj_list[i][select_random]])
            del(select_random)
        else:
            this.append([adj_list[i][0]])
    return this

In [17]:
def get_community(chromosome):
    
    class Graph:

        def __init__(self, V):
            self.V = V
            self.adj = [[] for i in range(V)]

        def DFSUtil(self, temp, v, visited):

            visited[v] = True

            temp.append(v)

            for i in self.adj[v]:
                if visited[i] == False:

                    temp = self.DFSUtil(temp, i, visited)
            return temp

        def addEdge(self, v, w):
            self.adj[v].append(w)
            self.adj[w].append(v)

        def connectedComponents(self):
            visited = []
            cc = []
            for i in range(self.V):
                visited.append(False)
            for v in range(self.V):
                if visited[v] == False:
                    temp = []
                    cc.append(self.DFSUtil(temp, v, visited))
            return cc



    g = Graph(node_number)
    for i in range(len(chromosome)):
        g.addEdge(i,chromosome[i][0])
    cc = g.connectedComponents()

    this_community = [0 for i in range(node_number)]
    for i in range(len(cc)):
        for j in cc[i]:
            this_community[j] = i+1

    return this_community

In [18]:
matrix = [[0 for i in range(node_number)] for i in range(node_number)]
for i in range(node_number):
    for j in range(len(adj_list[i])):
        b = adj_list[i][j]
        matrix[i][b] = 1
    
#list number of community coromosom
def fitness(list):
    
    sumation = 0
    m = edge_number
    for i in range(node_number):
        ki = sum(matrix[i])
        for j in range(node_number):
            if matrix[i][j]==1:
                A = 1
            else:
                A = 0
            kj = sum(matrix[j])

            delta = 0
            if list[i] == list[j]:delta = 1
            sumation = sumation + ((A)-((ki*kj)/(2*m)))*(delta)


    return(sumation/(2*edge_number))

In [19]:
#some parameter for run genetic algorithm insert here
#size of first population
normal_population = 40
max_iteration = 30
crossover_rate = 0.7
mutation_rate = 0.3
crossover_population = round((crossover_rate * normal_population))
mutation_population =round((mutation_rate * normal_population))
total = normal_population + crossover_population + mutation_population

In [20]:
def generate_population(adj_list,size):
    fitness_population = []
    population = []
    for i in range(size):
        deep = copy.deepcopy(adj_list)
        this_person = generate_chromosome(deep)
        population.append(this_person)
        community = get_community(this_person)
        this_fitness = fitness(community)
        fitness_population.append(this_fitness)
    
    return population,fitness_population

In [21]:
def mutation(chromosome):
    counter = 0
    while True:
        index = nr.randint(0,node_number-1)  
        if len(adj_list[index]) != 1:
            change = nr.randint(0,len(adj_list[index])-1)
            if chromosome[index] != [adj_list[index][change]]:
                chromosome[index] = [adj_list[index][change]]
                chromosome[adj_list[index][change]] = [index]
                break
        elif counter == 2*node_number:
            break
        counter+=1
        
    return chromosome

In [22]:
def crossover(male, female):
    
    breakpoint = nr.randint(1,node_number-1)
    child1 = male[:breakpoint] + female[breakpoint:]
    child2 = female[:breakpoint] + male[breakpoint:]
    return child1, child2

In [23]:
def cumulative_fitness(fitness,size):
    sum_fit = sum(fitness)
    cum = [0 for j in range(size)]
    for i in range(size):
        cum[i] = cum[i-1]+(fitness[i]/sum_fit)
        
    cum.append(1.0)
    
    return cum

In [24]:
def selection(cumulative_fitness):
    #roulette wheel
    def rotate():   
        here = np.random.rand()
        index = 0
        while(here>cumulative_fitness[index]):

            index+=1
        return index
    counter=0
    
    male = rotate()
    
    while True:

        female = rotate()
        counter += 1
        if male != female:
            break
    
    return male, female

In [25]:
#main script
this_population, this_fitness = generate_population(adj_list,normal_population)
top_fitness = [this_fitness[0]]
for i in range(max_iteration):
    cumulative = cumulative_fitness(this_fitness,normal_population)
    for i in range(crossover_population):
        male = female = 0
        counter = 0
        while male == female:
            counter += 1
            male, female = selection(cumulative)
            male, female = this_population[male], this_population[female]
            if male != female:
                break
            if counter==10:
                break
                
                
        child1, child2 = crossover(male, female)
        
        this_population.append(child1)
        this_population.append(child2)
        this_fitness.append(fitness(get_community(child1)))
        this_fitness.append(fitness(get_community(child2)))
        
    for i in range(mutation_population):
        rand = nr.randint(normal_population,total-1)
        mutant = mutation(this_population[rand])
        this_population.append(mutant)
        this_fitness.append(fitness(get_community(mutant)))
    
    this_fitness = []

    for i in range(len(this_population)):
        this_fitness.append(fitness(get_community(this_population[i])))
    
    
    final = sorted(zip(this_fitness,this_population), reverse=True)
    this_population_tmp = []
    this_fitness_tmp = []
    for i in range(normal_population):
        this_fitness_tmp.append(final[i][0])
        this_population_tmp.append(final[i][1])
        

    this_population = this_population_tmp
    this_fitness = this_fitness_tmp
    top_fitness.append(this_fitness[0])
    
    for i in range(1,10):
        print("Running","."*(10-i))             
    for i in range(1,10):
        print("Running","."*i)
  
    
result0 = this_population[0]
fitness0 = this_fitness[0]
print("result is:  ",result0)
final = get_community(result0)
print("communities detected:  ",final)
print("fitness is:  ",fitness0)


result is:   [[21], [21], [0], [0], [10], [16], [4], [3], [32], [33], [5], [0], [0], [0], [32], [32], [6], [1], [33], [0], [32], [1], [33], [27], [27], [31], [29], [23], [31], [32], [33], [24], [29], [14]]
communities detected:   [1, 1, 1, 1, 2, 2, 2, 1, 3, 3, 2, 1, 1, 1, 3, 3, 2, 1, 3, 1, 3, 1, 3, 4, 4, 4, 3, 4, 4, 3, 3, 4, 3, 3]
fitness is:   0.41978961209730375


In [26]:
#visulaize
from pyvis.network import Network
net = Network(height='750x',width='100%',bgcolor='#222222',font_color='white')
net.barnes_hut()
for i in range(node_number):
    net.add_node(i+1)
i=0
for ls in adj_list:
    for j in range(len(ls)):
        net.add_edge(i+1,ls[j]+1)
    i+=1


for i in range(node_number):
    if final[i]==1: flag = 'white'
    elif final[i]==2: flag = 'lime'
    elif final[i]==3: flag = 'cyan'
    elif final[i]==4: flag = 'yellow'
    net.add_node(100+i+1,color = flag)
i=0
for ls in adj_list:
    for j in range(len(ls)):
        net.add_edge(100+i+1,ls[j]+100+1)
    i+=1

net.show("genetic_main_visualizee.html")